In [1]:
!pip install langchain langchain_groq chromadb sentence_transformers
!pip install langchain-community pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 10.6 MB/s eta 0:0

In [2]:
from langchain_groq import ChatGroq
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
import uuid

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
from pdf_embed import collection

collection = collection

print(collection)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Collection(name=files)


In [4]:
response_ext = ["Hi Jimi. I would be you English buddy"]
user_ext = ["My name is Jimi"]

def generate_id(prefix):
    """Generate a unique ID using a prefix."""
    return f"{prefix}_{uuid.uuid4().hex}"  # Generates a random unique ID

In [25]:

# Initialize the ChatGroq client

#Chat

client = ChatGroq(
    model='llama3-8b-8192',
    groq_api_key="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",  # replace with your actual API key
    temperature=0
)

client2 = chromadb.PersistentClient()


collection2 = client2.get_or_create_collection(name = "history")
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def store(user_input, bot_response):


    """Stores Chat history in Chromadb"""
    documents =[user_input, bot_response]
    embedding2 = model.encode(documents)
    ids = [generate_id(user_input), generate_id(bot_response)]


    collection2.add(
        documents =documents,
        ids =ids,
        embeddings = embedding2
    )




def Retrieve(query):
    """Retrieves chat history from Chromadb"""
    query_embedding = model.encode(query)
    results = collection2.query(query_embeddings = query_embedding, n_results = 1)

    if results["documents"]:
            #print(results)  # Optional: See the full response structure
            # Return the first matching document
            return results["documents"][0]
    else:
        return None



def Retrieve_pdf(query):
  """Retrieve pdf contents and return results"""
  embed = model.encode(query)
  results = collection.query(query_embeddings = embed, n_results = 3)

  if results["documents"]:
    return results["documents"][0]
  else:
      return None





def get_response(user_input):
    """Generate bot response and use ChromaDB for context if available."""
    # Try to retrieve a relevant past conversation from the database
    relevant_response = Retrieve(user_input)
    relevant_response2 = Retrieve_pdf(user_input)
    #print(relevant_response2)

    if relevant_response:
        messages = [
            ("system", f"You are Pearl, a friendly and engaging English tutor who corrects grammatical errors and help improve english of its students. Do not introduce yourself again. Ensure to correct mistakes. Simply continue the conversation based on the user's input. Correct mistakes in grammar, lexis, and sentence structure when necessary. Keep the conversation flowing naturally while offering the corrections. The user’s previous response is there to maintain the flow, but do not repeat it. Use insights from previous messages to guide the conversation and provide relevant corrections. If the user says 'explain' without specifying what to explain, use {relevant_response} to provide an answer."),
            ("human", f"User: {user_input}\n Previous response: {relevant_response}")
        ]
    else:
        messages = [
            ("system", f"Your name is Pearl. You a friendly and engaging English tutor. Correct mistakes when necessary, and keep the conversation flowing naturally.  Start the conversation by introducing yourself but avoid introducing yourself more than once."),
            ("human", f"User: {user_input}")
        ]


    try:
      response = client.invoke(messages)
    #stores chat history
      store(user_input, str(response))

      return response
    except Exception as e:
        return f"Error: {str(e)}"


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
#game

import random


random_words = {
    "quick": ["fast", "rapid", "swift", "speedy", "hasty", "prompt", "expeditious", "nimble", "brisk", "fleet"],
    "bright": ["luminous", "shiny", "radiant", "vivid", "brilliant", "glowing", "gleaming", "sparkling", "dazzling", "vibrant"],
    "difficult": ["hard", "challenging", "tough", "arduous", "complicated", "complex", "strenuous", "formidable", "demanding", "tricky"],
    "beautiful": ["pretty", "attractive", "lovely", "gorgeous", "stunning", "handsome", "exquisite", "charming", "elegant", "radiant"],
    "angry": ["mad", "irate", "furious", "enraged", "irritated", "infuriated", "fuming", "livid", "wrathful", "outraged"],
    "lazy": ["idle", "sluggish", "inactive", "lethargic", "apathetic", "dormant", "indolent", "listless", "slothful", "unmotivated"],
    "happy": ["joyful", "content", "pleased", "cheerful", "delighted", "ecstatic", "satisfied", "elated", "blissful", "jovial"],
    "small": ["tiny", "petite", "mini", "minute", "microscopic", "compact", "slight", "modest", "little", "miniature"],
    "large": ["huge", "big", "gigantic", "massive", "immense", "colossal", "vast", "enormous", "grand", "tremendous"],
    "smart": ["intelligent", "clever", "bright", "sharp", "brilliant", "wise", "astute", "quick-witted", "ingenious", "knowledgeable"],
    "cold": ["chilly", "cool", "freezing", "icy", "frigid", "glacial", "nippy", "wintry", "crisp", "polar"],
    "warm": ["hot", "toasty", "comfortable", "balmy", "pleasant", "humid", "cozy", "lukewarm", "mild", "temperate"],
    "strong": ["sturdy", "powerful", "robust", "tough", "solid", "muscular", "resilient", "vigorous", "stalwart", "forceful"],
    "weak": ["frail", "fragile", "delicate", "feeble", "brittle", "fragile", "infirm", "powerless", "vulnerable", "unsteady"],
    "funny": ["humorous", "comical", "amusing", "hilarious", "witty", "jovial", "laughable", "entertaining", "fun", "lighthearted"],
    "sad": ["unhappy", "sorrowful", "downcast", "mournful", "despondent", "depressed", "blue", "melancholy", "glum", "disheartened"],
    "clean": ["tidy", "neat", "spotless", "pristine", "orderly", "immaculate", "organized", "uncluttered", "polished", "sanitary"],
    "dirty": ["filthy", "grimy", "messy", "soiled", "unclean", "muddy", "stained", "polluted", "sullied", "contaminated"],
    "brave": ["courageous", "valiant", "fearless", "gallant", "heroic", "bold", "intrepid", "daring", "audacious", "undaunted"],
    "calm": ["peaceful", "serene", "relaxed", "tranquil", "composed", "collected", "placid", "unruffled", "even-tempered", "cool-headed"]
}


'''def evaluate_synonym(word, user_word):
    # Sends prompt to OpenAI model to determine if user's word is a synonym
    messages = [
            ("system", f"Is {user_word} a synonym to {word}. Respond with '1' if it is yes and '0' if no")
      ]


    score = client.invoke(messages)

    return score == '1'

def synonym_game():
    print("\nWelcome to the Synonym Game!")
    print("I'll give you a word, and you need to type a synonym for it.")
    print("Type 'exit' to stop playing the game.\n")

    while True:

      word = random.choice(random_words)
      print(f"What's a synonym for '{word}'?")
      user_word = input("You: ").lower()



      if user_word == 'exit':
          print("Exiting the game. Thanks for playing!")
          break

      if evaluate_synonym(word, user_word):
          print("Correct! Well done!")
      else:
          print("Oops! That's not quite right. Try again!")'''


# Function to evaluate if the user's word is a synonym of the given word
def evaluate_synonym(word, user_word):
    user_word = user_word.lower()
    if user_word in random_words.get(word, []):
        return True
    return False

# Synonym game loop
def synonym_game():
    print("\nWelcome to the Synonym Game!")
    print("I'll give you a word, and you need to type a synonym for it.")
    print("Type 'exit' to stop playing the game.\n")

    while True:
        word = random.choice(list(random_words.keys()))
        print(f"What's a synonym for '{word}'?")
        user_word = input("You: ").strip().lower()

        if user_word == 'exit':
            print("Exiting the game. Thanks for playing!")
            break

        # Check if the user's word is a valid synonym
        if evaluate_synonym(word, user_word):
            print("Correct! Well done!")
        else:
            print(f"Oops! Some correct answers could be: {', '.join(random_words[word])}. Try again!")



In [29]:
def main():
    print("Welcome to the English Learning Chatbot!")
    print("Type 'exit' to end the conversation.\n")

    choice = input("Would you like to play a game or just chat? \nType 'game' to play \nType 'chat' to just chat: ").strip().lower()

    if choice == 'game':
        synonym_game()

    elif choice == 'chat':
        print("\nYou are now in chat mode! Type 'exit' to end the chat.\n")
        print("\nI am Pearl your personal AI english tutor")
        print("Please be specific with questions to enable me respond accordingly")
        while True:
            user_input = input("You: ").strip()

            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Get response from the bot
            bot_response = get_response(user_input)
            print(f"Bot: {bot_response.content}")

    else:
        print("Invalid choice. Please restart and type 'game' or 'chat'.")

if __name__ == "__main__":
    main()

Welcome to the English Learning Chatbot!
Type 'exit' to end the conversation.

Would you like to play a game or just chat? 
Type 'game' to play 
Type 'chat' to just chat: chat

You are now in chat mode! Type 'exit' to end the chat.


I am Pearl your personal AI english tutor
Please be specific with questions to enable me respond accordingly
You: Hello Peatl
Bot: I think there might be a small mistake there! It's actually "Pearl", not "Peatl". But no worries, I'm here to help!

So, what's on your mind? Do you have a specific question or topic you'd like to discuss?
You: Want talk English practice.
Bot: I'd love to practice some English conversation with you. Why don't we start with a topic? We could talk about your hobbies, interests, or even a recent event that happened in your life.
You: I love play ball
Bot: It sounds like you're excited about playing ball! However, I noticed that there's a small mistake in your sentence. Instead of "I love play ball", it should be "I love to play ba

In [28]:
#Pick a role play and tell the bot to play along